## 0. PSEUDOCODE / OVERVIEW

##### Prep data
Merge countries' hazard impact vectors into one gdf.

##### Spatial join to project ADMs
Get centroids of hazard gdf.
<br> For each ADM, spatial join ADM with centroids (contains).

##### Combine into dataframe
Provide unique field names.
<br> Table join on ADM3 code.

## 1. PREPARE WORKSPACE

### 1.1 Load all packages.

In [1]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from rasterstats import zonal_stats
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

In [2]:
# The usual directories
Project_Fd = os.getcwd()
Current_Fd = os.path.join(Project_Fd, 'Hazard')
Source_Fd = os.path.join(Current_Fd, 'Source', '2022')
Intermed_Fd = os.path.join(Current_Fd, 'Intermediate')

# Auxilliary sources
ADM_Fd = os.path.join(Project_Fd, 'ADM')
ADM_gpkg = os.path.join(ADM_Fd, 'Sahel_AdminBoundaries.gpkg')
CCDR_Fd = 'Q:\GIS\povertyequity\CCDR'

# Check paths
print('\n\n'.join([Project_Fd, Current_Fd, Source_Fd, Intermed_Fd, CCDR_Fd, ADM_Fd, ADM_gpkg]))

Q:\GIS\povertyequity\PTI_Sahel

Q:\GIS\povertyequity\PTI_Sahel\Hazard

Q:\GIS\povertyequity\PTI_Sahel\Hazard\Source\2022

Q:\GIS\povertyequity\PTI_Sahel\Hazard\Intermediate

Q:\GIS\povertyequity\CCDR

Q:\GIS\povertyequity\PTI_Sahel\ADM

Q:\GIS\povertyequity\PTI_Sahel\ADM\Sahel_AdminBoundaries.gpkg


## 2. PREP ADMIN LAYERS

### 2.1 CCDR-to-PTI dictionary

Note: this dictionary was made manually in QGIS based on admin boundary alignments between the incongruent admin area datasets.

CCDR admin area count: (remember that one country is missing in each admin set)
<br>ADM2: 182
<br>ADM3: 1329

PTI admin area count:
<br>ADM2: 275
<br>ADM3: 1433

#### CCDR-PTI dictionary

In [3]:
CCDR_ADM2 = pd.DataFrame(gpd.read_file(
    os.path.join(Intermed_Fd, 'CCDR_ADM2_consolidated.shp')))[['CCDR', 'ADM2_CODE', 'ADM2_s2', 'ADM2_s3', 'ADM2_s4']]

CCDR_ADM3 = pd.DataFrame(gpd.read_file(
    os.path.join(Intermed_Fd, 'CCDR_ADM3_consolidated.shp')))[['CCDR', 'ADM3_CODE', 'ADM3_s2', 'ADM3_s3']]

print(CCDR_ADM2.info(), CCDR_ADM3.info(), '\n\n', CCDR_ADM2.sample(10), '\n\n', CCDR_ADM3.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CCDR       182 non-null    int64 
 1   ADM2_CODE  182 non-null    object
 2   ADM2_s2    23 non-null     object
 3   ADM2_s3    7 non-null      object
 4   ADM2_s4    2 non-null      object
dtypes: int64(1), object(4)
memory usage: 7.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329 entries, 0 to 1328
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CCDR       1329 non-null   float64
 1   ADM3_CODE  1329 non-null   object 
 2   ADM3_s2    4 non-null      object 
 3   ADM3_s3    1 non-null      object 
dtypes: float64(1), object(3)
memory usage: 41.7+ KB
None None 

      CCDR ADM2_CODE ADM2_s2 ADM2_s3 ADM2_s4
70   5002    NE0502    None    None    None
158  1601    TD1601  TD1604    None    None
4    4601    BF0201    None  

#### Pivot longer: ADM2

In [4]:
Dict_ADM2 = pd.melt(CCDR_ADM2, id_vars=['CCDR'], value_vars=[col for col in CCDR_ADM2 if col.startswith('ADM2')], 
                    value_name='PTI')
Dict_ADM2 = Dict_ADM2.loc[Dict_ADM2['PTI'].notnull()][['CCDR', 'PTI']]
Dict_ADM2

,CCDR,PTI
0,5503,BF1103
1,4803,BF0403
2,5603,BF1203
3,5402,BF1002
4,4601,BF0201
...,...,...
531,2101,TD2103
533,2201,TD2201
535,2301,TD2004
713,2101,TD2104


In [5]:
print('Number of PTI polygons associated with more than one CCDR polygon: ', 
      (len(Dict_ADM2.index.unique()) - len(Dict_ADM2['CCDR'].unique())))
print('Number of CCDR polygons associated with more than one PTI polygon: ', 
      (len(Dict_ADM2.index.unique()) - len(Dict_ADM2['PTI'].unique())))

Number of PTI polygons associated with more than one CCDR polygon:  41
Number of CCDR polygons associated with more than one PTI polygon:  0


#### Pivot longer: ADM3

In [6]:
Dict_ADM3 = pd.melt(CCDR_ADM3, id_vars=['CCDR'], value_vars=[col for col in CCDR_ADM3 if col.startswith('ADM3')], 
                    value_name='PTI')
Dict_ADM3 = Dict_ADM3.loc[Dict_ADM3['PTI'].notnull()][['CCDR', 'PTI']]
Dict_ADM3

,CCDR,PTI
0,550301.0,BF110301
1,480301.0,BF040301
2,560301.0,BF120301
3,540201.0,BF100201
4,540202.0,BF100202
...,...,...
2179,60301.0,ML090401
2199,80401.0,ML040402
2266,100301.0,ML020403
2275,70307.0,ML020304


In [7]:
print('Number of PTI polygons associated with more than one CCDR polygon: ', 
      (len(Dict_ADM3.index.unique()) - len(Dict_ADM3['CCDR'].unique())))
print('Number of CCDR polygons associated with more than one PTI polygon: ', 
      (len(Dict_ADM3.index.unique()) - len(Dict_ADM3['PTI'].unique())))

Number of PTI polygons associated with more than one CCDR polygon:  16
Number of CCDR polygons associated with more than one PTI polygon:  20


## 2. PREP RISK DATA

### 2.1 Clean up risk layers

#### Load all indicators as separate ADM2 and ADM3 objects.

In [ ]:
BFA_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "BFA*ADM2*"))]
NER_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "NER*ADM2*"))]
TCD_ADM2 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "TCD*ADM2*"))]

BFA_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "BFA*ADM3*"))]
MLI_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "MLI*ADM3*"))]
NER_ADM3 = [pd.DataFrame(gpd.read_file(f, layer=os.path.basename(f).replace('.gpkg', ''))).drop(columns='geometry') 
            for f in glob.glob(os.path.join(Source_Fd, "NER*ADM3*"))]

In [ ]:
print(len(BFA_ADM2), len(BFA_ADM3))
print(len(MLI_ADM3))
print(len(NER_ADM2), len(NER_ADM3))
print(len(TCD_ADM2))

#### Drought and flood have some inconsistent naming. Rename to match before concatenating.

In [ ]:
dropADM = ['ADM0_CODE', 'ADM0_NAME', 'ADM1_CODE', 'ADM1_NAME', 'ADM2_NAME', 'ADM3_NAME']

ListofLists = [BFA_ADM2, NER_ADM2, TCD_ADM2, BFA_ADM3, MLI_ADM3, NER_ADM3]

for dfList in ListofLists:
    i = 0
    for df in dfList:
        if 'S1_30_mean' in df.columns:
            dfList[i] = df.rename(columns={'S1_30_mean':'S1_30p_mean'})
        if 'S1_50_mean' in df.columns:
            dfList[i] = df.rename(columns={'S1_50_mean':'S1_50p_mean'})
        if 'EAE' in df.columns:
            dfList[i] = df.rename(columns={'EAE':'EAE_agri'})
            
            
        # Let's also remove the extra admin areas while we're at it. Otherwise the pd.concat() function will have trouble.
        try:
            dfList[i] = df.drop(columns=df.columns.intersection(dropADM), axis=1)
        except:
            pass
        if 'ADM3_CODE' in df.columns:
            try:
                dfList[i] = df.drop(columns='ADM2_CODE', axis=1) # If the df is ADM3 features, drop the ADM2 code.
            except:
                pass
        print(dfList[i].info())
        i = i + 1

### 2.2 Merge and concatenate indicators

#### Merge indicators of same country

In [ ]:
Merged = []
for dfList in ListofLists:
    Merged = Merged + [pd.concat(dfList, axis=1).reset_index()]

In [ ]:
Merged[5]

#### Concatenate countries together

In [ ]:
Indic_A2 = pd.concat(Merged[0:2], axis=1)
Indic_A3 = pd.concat(Merged[3:5], axis=1)
Indic_A2

In [ ]:
Indic_A3

In [ ]:
# PTI and CCDR admin fields are the same label. Changing to distinguish from one another.
Indic_A2 = Indic_A2.rename(columns={'ADM2_CODE':'CCDR'})
Indic_A3 = Indic_A3.rename(columns={'ADM3_CODE':'CCDR'})

### 2.2 Merge with CCDR ADMs

In [ ]:
CCDR2 = CCDR_ADM2[['CCDR']].merge(Indic_A2, how='left', on='CCDR')
CCDR3 = CCDR_ADM3[['CCDR']].merge(Indic_A3, how='left', on='CCDR')

print(CCDR2.info(), CCDR2.sample(10))

In [ ]:
Indic3 = FL_ADM3 + [CCDR_ADM3] + HS_ADM3 + DR_ADM3 + LS_ADM3
CCDR3 = 
CCDR3 = pd.concat(Indic3, axis=1)
CCDR3

#### Concatenate each group.

In [ ]:
HZ_A2 = {'DR':pd.concat(DR_ADM2, ignore_index=True), 
         'HS':pd.concat(HS_ADM2, ignore_index=True), 
         'FL':pd.concat(FL_ADM2, ignore_index=True),
         'LS':pd.concat(LS_ADM2, ignore_index=True)}
for k in HZ_A2:
    HZ_A2[k] = HZ_A2[k].rename(columns={'ADM2_CODE':'CCDR'}) # Rename admin area field to match CCDR-PTI dictionary.
    print('\n\n', k + ": ", HZ_A2[k]['ADM0_NAME'].unique())
    print('\n\n', HZ_A2[k].info(), '\n\n', HZ_A2[k].sample(10))

In [ ]:
HZ_A3 = {'DR':pd.concat(DR_ADM3, ignore_index=True), 
         'HS':pd.concat(HS_ADM3, ignore_index=True), 
         'FL':pd.concat(FL_ADM3, ignore_index=True),
         'LS':pd.concat(LS_ADM3, ignore_index=True)}
for k in HZ_A3:
    HZ_A3[k] = HZ_A3[k].rename(columns={'ADM3_CODE':'CCDR'})
    print('\n\n', k + ": ", HZ_A3[k]['ADM0_NAME'].unique())
    print('\n\n', HZ_A3[k].info(), '\n\n', HZ_A3[k].sample(10))

#### Remove unnecessary fields.

In [ ]:
# Remove excess variables.
Indicators = ['S1_30p_mean', 'S1_50p_mean', # DR
              'EAE_C4', 'EAE_C4%', # HS
              'builtup_EAI', 'builtup_EAI%', 'pop_EAI', 'pop_EAI%', 'EAE_agri', # FL
              'builtup_tot_exposed', 'pop_tot_exposed' # LS
              ]

Fields = Indicators + ['CCDR']

In [ ]:
Indic_A2 = HZ_A2.copy()
Indic_A3 = HZ_A3.copy()

# Overwrite the values in the dictionary with the updated, smaller dfs.
for k in HZ_A2:
    Indic_A2[k] = HZ_A2[k][HZ_A2[k].columns.intersection(Fields)]
for k in HZ_A3:
    Indic_A3[k] = HZ_A3[k][HZ_A3[k].columns.intersection(Fields)]

Indic_A2

In [ ]:
Indic_A3

#### Rename fields to include hazard type.

In [ ]:
for k in Indic_A2:
    Indic_A2[k].columns = [k + '_' + col if col in Indicators else col for col in Indic_A2[k].columns]
Indic_A2

In [ ]:
for k in Indic_A3:
    Indic_A3[k].columns = [k + '_' + col if col in Indicators else col for col in Indic_A3[k].columns]
Indic_A3

### 2.2 Merge risk indicators by CCDR ADM

#### Merge indicators into one df per ADM type now that hazard indicators have unique labels.

In [ ]:
# No need for dictionary keys anymore. Let's simplify.
Indic_A2 = list(Indic_A2.values())
Indic_A3 = list(Indic_A3.values())
print(len(Indic_A2), len(Indic_A3))

In [ ]:
# Tried doing this as a for loop (for k, prevk in zip()), but was struggling.
Indic_ADM2 = Indic_A2[0].merge(Indic_A2[1], on='CCDR', how='outer')
Indic_ADM2 = Indic_ADM2.merge(Indic_A2[2], on='CCDR', how='outer')
Indic_ADM2 = Indic_ADM2.merge(Indic_A2[3], on='CCDR', how='outer')

Indic_ADM3 = Indic_A3[0].merge(Indic_A3[1], on='CCDR', how='outer')
Indic_ADM3 = Indic_ADM3.merge(Indic_A3[2], on='CCDR', how='outer')
Indic_ADM3 = Indic_ADM3.merge(Indic_A3[3], on='CCDR', how='outer')

In [ ]:
Indic_ADM2.sample(20)

In [ ]:
Indic_ADM3.sample(20)

## 4. MATCH RISK INDICATORS WITH PTI ADMIN AREAS

### 4.1 Categorize indicators by eligible aggregation or disaggregation method.

If splitting a hazard ADM into 2+ project ADMs, AND if indicator does not depend on the polygon size or amount (intensive):
<br> Indicator unchanged, applied to each ADM.

If splitting a hazard ADM into 2+ project ADMs, AND if indicator is dependent on the polygon size or amount (extensive):
<br> No data applied to each ADM.

If dissolving 2+ hazard ADMs into a project ADM, AND if indicator is intensive:
<br> No data applied to each ADM.

If dissolving 2+ hazard ADMs into a project ADM, AND if indicator is extensive:
<br> -- If indicator is count, apply sum.
<br> -- If indicator is average, apply no data.
<br> -- If indicator is max, apply max.

Drought percent: INTENSIVE, AVERAGE
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

Landslide: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data. 

Flood: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data.

Flood percent: INTENSIVE, PROPORTION
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

Heat stress: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data.

Heat stress percent: INTENSIVE, PROPORTION
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

In [ ]:
print(Indic_ADM2.info(), '\n', Indic_ADM3.info())

In [ ]:
# Take the Indicators list and assign intensive or extensive.
Intensive = ['DR_S1_30p_mean', 'DR_S1_50p_mean', # DR
              'HS_EAE_C4%', # HS
              'FL_builtup_EAI%', 'FL_pop_EAI%' # FL
              # LS
            ]

Extensive = [# DR
              'HS_EAE_C4', # HS
              'FL_builtup_EAI', 'FL_pop_EAI', 'FL_EAE_agri', # FL
              'LS_builtup_tot_exposed', 'LS_pop_tot_exposed' # LS
             ]

### 4.2 Group-by aggregations

#### Table join risk indicators onto project ADMs.

In [ ]:
ADM2 = CCDR_ADM2.merge(Indic_ADM2, on='CCDR', how='left')
ADM3 = CCDR_ADM3.merge(Indic_ADM3, on='CCDR', how='left')
print(ADM2.info(), ADM3.info())

#### Identify duplicates and mark them in the expanded dataset.

In [ ]:
# subset rows that have duplicate PTI ADM: dupePTI
dupePTI_A2 = ADM2[ADM2.duplicated(['ADM2_CODE'], keep='first')]
dupePTI_A2 = list(dupePTI_A2['ADM2_CODE'])

dupePTI_A3 = ADM3[ADM3.duplicated(['ADM3_CODE'], keep='first')]
dupePTI_A3 = list(dupePTI_A3['ADM3_CODE'])

In [ ]:
# subset rows that have duplicate CCDR ADM: dupeCCDR
dupeCCDR_A2 = ADM2[ADM2.duplicated(['CCDR'], keep='first')]
dupeCCDR_A2 = list(dupeCCDR_A2['CCDR'])

dupeCCDR_A3 = ADM3[ADM3.duplicated(['CCDR'], keep='first')]
dupeCCDR_A3 = list(dupeCCDR_A3['CCDR'])

In [ ]:
ADM2['dupeCCDR'] = 0 # Default value will be zero
ADM2['dupePTI'] = 0
ADM2.loc[ADM2.CCDR.isin(dupeCCDR_A2), 'dupeCCDR'] = 1
ADM2.loc[ADM2.ADM2_CODE.isin(dupePTI_A2), 'dupePTI'] = 1

ADM3['dupeCCDR'] = 0
ADM3['dupePTI'] = 0
ADM3.loc[ADM3.CCDR.isin(dupeCCDR_A3), 'dupeCCDR'] = 1
ADM3.loc[ADM3.ADM3_CODE.isin(dupePTI_A3), 'dupePTI'] = 1

print(ADM2.sample(10), ADM3.sample(10))

In [ ]:
# Check to make sure there are 2 unique values for the dupe fields.
print(ADM2.nunique(axis=0), ADM3.nunique(axis=0))

In [ ]:
# We can drop CCDR ADM field now
ADM2 = ADM2.drop(columns='CCDR', axis=1)
ADM3 = ADM3.drop(columns='CCDR', axis=1)

#### Group-by PTI ADMs and apply appropriate aggregation or null setting

In [ ]:
# Now we can work with just the PTI ADM features.
ADM2_Int = ADM2.drop(columns=Extensive).groupby(
    'ADM2_CODE', as_index=False).first() # Keep original value

ADM2_Ext = ADM2.drop(columns=Intensive).groupby(
    'ADM2_CODE', as_index=False).sum() # Sum these counts
ADM2_Int.info()

In [ ]:
ADM3_Int = ADM3.drop(columns=Extensive).groupby(
    'ADM3_CODE', as_index=False).first() # Keep original value

ADM3_Ext = ADM3.drop(columns=Intensive).groupby(
    'ADM3_CODE', as_index=False).sum() # Sum these counts
ADM3_Int.info()

In [ ]:
# Change extensive variables to None if there were multiple PTI ADMs
for col in Extensive:
    ADM2_Ext.loc[ADM2_Ext.dupePTI > 0, col] = None
# Change intensive variables to None if there were multiple CCDR ADMs
for col in Intensive:
    ADM2_Int.loc[ADM2_Int.dupeCCDR > 0, col] = None

In [ ]:
for col in Extensive:
    ADM3_Ext.loc[ADM3_Ext.dupePTI > 0, col] = None
for col in Intensive:
    ADM3_Int.loc[ADM3_Int.dupeCCDR > 0, col] = None

In [ ]:
# Merge intensive and extensive variables into single df and save to file.
ADM2_PTI = ADM2_Int.merge(ADM2_Ext, on='ADM2_CODE', how='outer')
ADM2_PTI = ADM2_PTI.loc[:, ~ADM2_PTI.columns.str.contains('dupe')]
ADM3_PTI = ADM3_Int.merge(ADM3_Ext, on='ADM3_CODE', how='outer')
ADM3_PTI = ADM3_PTI.loc[:, ~ADM3_PTI.columns.str.contains('dupe')]
print(ADM2_PTI.info(), ADM3_PTI.info())

In [ ]:
ADM2_PTI.sample(20)

In [ ]:
ADM3_PTI.sample(20)

### 2.1 Concatenate countries' CCDR admin areas into a parent object

In [ ]:
FileNames_ADM2 = [''.join([ISO, '_FL_ADM2_pop_EAI']) for ISO in ['BFA', 'NER', 'TCD']]
FileNames_ADM3 = [''.join([ISO, '_FL_ADM3_pop_EAI']) for ISO in ['BFA', 'MLI', 'NER']]

# ADM2
CCDR_ADM2 = [gpd.read_file(os.path.join(Source_Fd, ''.join([FileName, '.gpkg'])), layer=FileName, driver='GPKG') for FileName in FileNames_ADM2]

# ADM3
CCDR_ADM3 = [gpd.read_file(os.path.join(Source_Fd, ''.join([FileName, '.gpkg'])), layer=FileName, driver='GPKG') for FileName in FileNames_ADM3]

print([Item.info() for Item in CCDR_ADM2], [Item.info() for Item in CCDR_ADM3])

In [ ]:
print('Match all projections for concatenation.')
for prevf, f in zip(CCDR_ADM2, CCDR_ADM2[1:]):
    print('Checking...')
    if f.crs != prevf.crs:
        try:
            f.to_crs(prevf.crs)
            print('Reprojecting to match previous.')
        except:
            pass
    else:
        print('Matches with previous.')
        
print('\n\nFinal CRS list:')
for f in CCDR_ADM2:
    print(f.crs)

In [ ]:
print('Match all projections for concatenation.')
for prevf, f in zip(CCDR_ADM3, CCDR_ADM3[1:]):
    print('Checking...')
    if f.crs != prevf.crs:
        try:
            f.to_crs(prevf.crs)
            print('Reprojecting to match previous.')
        except:
            pass
    else:
        print('Matches with previous.')
        
print('\n\nFinal CRS list:')
for f in CCDR_ADM3:
    print(f.crs)

In [ ]:
CCDR_A2 = pd.concat(CCDR_ADM2, ignore_index=True)
print(CCDR_A2.info(), '\n\n', CCDR_A2['ADM0_NAME'].unique(), '\n\n', CCDR_A2.head(10))

In [ ]:
CCDR_A3 = pd.concat(CCDR_ADM3, ignore_index=True)
print(CCDR_A3.info(), '\n\n', CCDR_A3['ADM0_NAME'].unique(), '\n\n', CCDR_A3.head(10))

In [ ]:
# Drop excess variables and rename the ADM codes since their names are identical to the PTI.
CCDR_A2 = CCDR_A2[['ADM0_CODE', 'ADM2_CODE', 'geometry']]
CCDR_A2 = CCDR_A2.rename(columns={'ADM0_CODE':'ISO', 'ADM2_CODE':'ADM2_CCDR'})

CCDR_A3 = CCDR_A3[['ADM0_CODE', 'ADM3_CODE', 'geometry']]
CCDR_A3 = CCDR_A3.rename(columns={'ADM0_CODE':'ISO', 'ADM3_CODE':'ADM3_CCDR'})

print(CCDR_A2.info(), CCDR_A3.info())